<a href="https://colab.research.google.com/github/stavis1/GST_colloquium/blob/main/GST_colloquium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [5]:
!pip install brain-isotopic-distribution
!pip install multiprocesspandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.6/126.6 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00


In [6]:
import scipy
import pandas as pd
import numpy as np
from brainpy import isotopic_variants
import numba
from functools import cache
import multiprocesspandas